<a href="https://colab.research.google.com/github/drdssouza/Recomendacao-Series/blob/main/RecomendacaoSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Código fazendo a limpeza dos dados na API do TMDB para séries de Crime, Guerra e Mistério

Importando as bibliotecas necessárias

In [1]:
import requests
import json
import math

Definição das funções

In [2]:
def obter_resposta_api(url):
    resposta = requests.get(url)
    return resposta.json()

def obter_mapa_id_para_nome_genero(api_key):
    url_generos = f'https://api.themoviedb.org/3/genre/tv/list?api_key={api_key}&language=pt-BR'
    dados_generos = obter_resposta_api(url_generos)

    mapa_id_para_nome = {}

    for genero in dados_generos['genres']:
        mapa_id_para_nome[genero['id']] = genero['name']

    return mapa_id_para_nome

def obter_detalhes_serie(api_key, id_serie):
    url_detalhes_serie = f'https://api.themoviedb.org/3/tv/{id_serie}?api_key={api_key}&language=pt-BR'
    return obter_resposta_api(url_detalhes_serie)

def filtrar_series(api_key, todos_resultados, generos_desejados):
    mapa_id_para_nome = obter_mapa_id_para_nome_genero(api_key)
    series_filtradas = []

    for serie in todos_resultados:
        ids_generos = serie['genre_ids']

        if any(id_genero in generos_desejados for id_genero in ids_generos):
            nomes_generos_desejados_str = ','.join(mapa_id_para_nome[id_genero] for id_genero in ids_generos if id_genero in generos_desejados)

            id_serie = str(serie['id'])
            detalhes_serie = obter_detalhes_serie(api_key, id_serie)

            empresas_producao = [empresa['name'] for empresa in detalhes_serie.get('production_companies', [])]
            empresas_producao_str = ', '.join(empresas_producao)

            paises_producao_str = ', '.join(detalhes_serie.get('origin_country', []))

            numero_temporadas = detalhes_serie.get('number_of_seasons', '')
            numero_episodios = detalhes_serie.get('number_of_episodes', '')
            media_votos = detalhes_serie.get('vote_average', '')

            if paises_producao_str and empresas_producao_str:
                series_filtradas.append({
                    'id': id_serie,
                    'nome_original': serie['original_name'],
                    'generos': nomes_generos_desejados_str,
                    'data_estreia': serie['first_air_date'],
                    'popularidade': serie['popularity'],
                    'empresas_producao': empresas_producao_str,
                    'paises_producao': paises_producao_str,
                    'numero_temporadas': numero_temporadas,
                    'numero_episodios': numero_episodios,
                    'media_votos': media_votos
                })

    return series_filtradas

def salvar_lote_series(lote_series, indice_lote):
    nome_arquivo = f'series_filtradas_{indice_lote + 1}.json'
    conteudo_arquivo = json.dumps(lote_series)

    with open(nome_arquivo, 'w') as f:
        f.write(conteudo_arquivo)


Definindo a função MAIN
- Coloque sua API no campo api_key
- Definda a quantidade de páginas a fazer a busca no campo total_paginas


In [3]:
def main():
    api_key = 'df17b17c4e452ad1676517f6fa195825'
    url = f'https://api.themoviedb.org/3/tv/popular?language=pt-BR&page=1&api_key={api_key}'

    resposta = obter_resposta_api(url)
    todos_resultados = resposta['results']

    pagina = 1
    total_paginas = 500

    while pagina < total_paginas:
        pagina += 1
        url_atualizada = url.replace('page=1', f'page={pagina}')
        resposta = obter_resposta_api(url_atualizada)
        todos_resultados.extend(resposta['results'])

    generos_desejados = [80, 10752, 9648]  # IDs dos gêneros Crime, Guerra e Mistério

    series_filtradas = filtrar_series(api_key, todos_resultados, generos_desejados)

    tamanho_lote = 100
    total_lotes = math.ceil(len(series_filtradas) / tamanho_lote)

    for indice_lote in range(total_lotes):
        inicio_lote = indice_lote * tamanho_lote
        fim_lote = inicio_lote + tamanho_lote
        lote_series = series_filtradas[inicio_lote:fim_lote]
        salvar_lote_series(lote_series, indice_lote)

Com a API e o total de páginas definidas ali em cima, rode o código abaixo:

In [4]:
if __name__ == '__main__':
    main()

# Sistema de Recomendação de série com Machine Learning

Importação das bibliotecas

In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

Definindo as Funções

In [1]:
def carregar_series_from_json():
    series = []
    for i in range(15):
        filename = f'series_filtradas_{i + 1}.json'
        with open(filename, 'r') as f:
            series_data = json.load(f)
            series.extend(series_data)
    return series

def preprocessar_dados(series):
    df_series = pd.DataFrame(series)
    if 'nome_original' in df_series.columns:
        df_series['nome_preprocessado'] = df_series['nome_original'].apply(lambda x: simple_preprocess(x))
    return df_series

def treinar_modelo(series):
    nome_preprocessado = series['nome_preprocessado']
    model = Word2Vec(nome_preprocessado, vector_size=100, window=5, min_count=1, workers=4)
    return model

def recomendar_serie(nome_serie, df_series, model, n=5):
    nome_preprocessado = simple_preprocess(nome_serie)
    nome_embedding = sum(model.wv[palavra] for palavra in nome_preprocessado if palavra in model.wv) / len(nome_preprocessado)

    embeddings = model.wv.vectors
    similaridades = cosine_similarity([nome_embedding], embeddings)
    indices_similares = similaridades.argsort()[0][::-1][:n]

    # Verificar se os índices estão dentro dos limites do conjunto de dados
    indices_validos = [idx for idx in indices_similares if idx < len(df_series)]

    recomendacoes = df_series.iloc[indices_validos]
    return recomendacoes[['nome_original', 'generos', 'popularidade']]

def salvar_recomendacoes(recomendacoes):
    for i, recomendacao in enumerate(recomendacoes):
        nome_arquivo = f'recomendacao_{i+1}.txt'
        with open(nome_arquivo, 'w') as f:
            f.write(recomendacao.to_string(index=False))

def main():
    series = carregar_series_from_json()
    df_series = preprocessar_dados(series)
    if df_series.empty:
        print("Erro: Não foi possível carregar os dados das séries.")
        return
    model = treinar_modelo(df_series)

    recomendacoes_total = []

    while True:
        nome_serie_usuario = input("Digite o nome de uma série que você gosta: ")
        recomendacoes = recomendar_serie(nome_serie_usuario, df_series, model)

        if recomendacoes.empty:
            print("Não foram encontradas recomendações para a série fornecida.")
            continue

        print(recomendacoes)
        recomendacoes_total.append(recomendacoes)

        continuar = input("\nDeseja continuar? (S para SIM, qualquer tecla para sair): ")
        if continuar.lower() != 's':
            break

    salvar = input("\nDeseja salvar as recomendações em arquivos .txt? (S para SIM, qualquer tecla para continuar): ")
    if salvar.lower() == 's':
        salvar_recomendacoes(recomendacoes_total)

    print(" ="*50,"\nObrigado por utilizar meu código!\nRealizado apenas para fins educativos e estudos adicionais sobre Machine Learning.\n","Feito durante o programa de bolsas da Compassuol\n", " ="*50,"\nCriado por Eduardo Schrotke.")


Executando o código

In [12]:
if __name__ == '__main__':
    main()

Digite o nome de uma série que você gosta: Desesperate Housewives


ValueError: ignored